In [5]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
import cv2
import keras
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from glob import glob
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
import tensorflow as tf
from keras import Sequential

from keras.layers import Dense, LSTM, \
    Flatten, TimeDistributed, Conv2D, Dropout

In [6]:
main_dir = 'C:/e/CRUPA/MISHA_heap_v2/'
main_dir_Anton = 'C:/e/CRUPA/ANTON_heap/'

# DATA GENERATOR

In [7]:

img_shape = (256, 256)
images_mish = sorted(os.listdir(main_dir))
images_anton = sorted(os.listdir(main_dir_Anton))
#images_misha = os.listdir(main_dir_Mish)
images = images_mish + images_anton 

images_info_df = pd.DataFrame(columns=['image_name', 'mass', 'food_type'])
for ind, img in enumerate(images):
    items = img.split('_')
    file = img.split('_')[2]
    #print(file[0:2])
    if file[0:2] == 'mv':
      images_info_df.loc[ind] = [main_dir + img, int(items[0]), items[1]]
    elif file[0:2] == 'av':
      images_info_df.loc[ind] = [main_dir_Anton + img, int(items[0]), items[1]]
    
#images_info_df['image_name'] = sorted(images_info_df['image_name'])
mass_list = list(images_info_df['mass'])  
mass_list = np.array(mass_list)

In [8]:
images_list = []
images_batch = []
mass_list = []
mass_batch = []
j = 0
for i, file in enumerate(images_info_df['image_name']):
  #print(i,file)
  if j < 5:
    j += 1
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_shape)
    images_batch.append(img)
    mass_batch.append(images_info_df['mass'][i])
    
  else:
    j = 0 
    images_list.append(images_batch)
    mass_list.append(int(images_info_df['mass'][i])) 
    images_batch = [] 
    mass_batch = []

images_list = np.array(images_list)
mass_list = np.array(mass_list)    

print(mass_list)

[100 100 100 100 100 100 100 100 100 100  10  10  10  10  10  10  10  10
  10  10 110 110 110 110 110 110 110 110 110 110 130 130 130 130 130 130
 130 130 130 130 140 140 140 140 140 140 140 140 140 140 150 150 150 150
 150 150 150 150 150 150 160 160 160 160 160 160 160 160 160 160 170 170
 170 170 170 170 170 170 170 170 180 180 180 180 180 180 180 180 180 180
 190 190 190 190 190 190 190 190 190 190 200 200 200 200 200 200 200 200
 200 200  20  20  20  20  20  20  20  20  20  20  30  30  30  30  30  30
  30  30  30  30  40  40  40  40  40  40  40  40  40  40  50  50  50  50
  50  50  50  50  50  50  60  60  60  60  60  60  60  60  60  60  70  70
  70  70  70  70  70  70  70  70  80  80  80  80  80  80  80  80  80  80
  90  90  90  90  90  90  90  90  90  90 100 100  10  10 110 110 120 120
 130 130 140 140 150 150 160 160 170 170 180 180 190 190 200 200  20  30
  30  40  40  50  50  60  60  70  70  80  90  90]


In [9]:
print(len(mass_list))

228


# MODEL

In [10]:
import keras
from keras.layers import Dense, LSTM, \
    Flatten, TimeDistributed, Conv2D, Dropout
from keras import Sequential
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import MobileNet,ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
# create a VGG16 "model", we will use
# image with shape (224, 224, 3)
base_model=ResNet50( weights='imagenet',include_top=False,input_shape=(256,256,3))
# vgg = VGG16(
#     include_top=False,
#     weights='imagenet',
#     input_shape=(224, 224, 3)
# )

for layer in base_model.layers[:-4]:
    layer.trainable = False

model = Sequential()

model.add(
    TimeDistributed(base_model, input_shape=(5, 256, 256, 3))
)

model.add(
    TimeDistributed(
        #Flatten()
        GlobalAveragePooling2D()
    )
)
model.add(LSTM(256, activation='relu', return_sequences=False))

model.add(Dense(64, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1, activation='linear'))
model.compile(optimizer = 'adam', loss='mean_absolute_error')

In [17]:
model = keras.models.load_model('C:/e/CRUPA/main_model/model_TIMEDIST_Resnet.h5')

# PREDICT 

In [23]:
y_pred = []
y_true = []
for i in range(140,200,3):
    
    img_sample = images_list[i]
    img_sample = np.expand_dims(img_sample, axis=0)
    pred = model.predict(img_sample)
    y_pred.append(pred)
    y_true.append(images_info_df['mass'][i])
    print(images_info_df['mass'][i], '-true')
    print(pred, '- predict')

110 -true
[[55.185642]] - predict
110 -true
[[51.60836]] - predict
110 -true
[[59.33707]] - predict
110 -true
[[56.850883]] - predict
110 -true
[[69.82325]] - predict
110 -true
[[72.218636]] - predict
110 -true
[[67.001915]] - predict
110 -true
[[88.83924]] - predict
110 -true
[[86.15855]] - predict
110 -true
[[82.86712]] - predict
110 -true
[[91.49111]] - predict
110 -true
[[88.86913]] - predict
110 -true
[[97.54469]] - predict
110 -true
[[102.12397]] - predict
130 -true
[[126.31305]] - predict
130 -true
[[109.33183]] - predict
130 -true
[[115.46587]] - predict
130 -true
[[185.68062]] - predict
130 -true
[[192.45717]] - predict
130 -true
[[152.58195]] - predict


In [24]:
df = pd.DataFrame({'y_predict':y_pred, 'y_true':y_true})
df.to_csv('C:/e/CRUPA/table3.csv')

In [25]:
res_df = pd.read_csv('C:/e/CRUPA/table3.csv', sep=',')
res_df['y_predict'] = res_df['y_predict'].apply(lambda x: round(float(x.strip('][')), 1))
res_df['error'] = round(res_df['y_predict'] - res_df['y_true'], 1)
res_df['error_in_%'] = round((res_df['y_predict'] - res_df['y_true']) * 100 / res_df['y_true'], 0)

In [26]:
res_df['error_in_%']

0    -50.0
1    -53.0
2    -46.0
3    -48.0
4    -37.0
5    -34.0
6    -39.0
7    -19.0
8    -22.0
9    -25.0
10   -17.0
11   -19.0
12   -11.0
13    -7.0
14    -3.0
15   -16.0
16   -11.0
17    43.0
18    48.0
19    17.0
Name: error_in_%, dtype: float64

# MODEL 2

In [ ]:
images_list = []
images_batch = []
mass_list = []
mass_batch = []
j = 0
img_shape = (224,224)
for i, file in enumerate(images_info_df['image_name']):
  #print(i,file)
  if j < 5:
    j += 1
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_shape)
    images_batch.append(img)
    mass_batch.append(images_info_df['mass'][i])
    
  else:
    j = 0 
    images_list.append(images_batch)
    mass_list.append(int(images_info_df['mass'][i])) 
    images_batch = [] 
    mass_batch = []

images_list = np.array(images_list)
mass_list = np.array(mass_list)    

print(mass_list)

In [ ]:
import keras
from keras.layers import Dense, LSTM, \
    Flatten, TimeDistributed, Conv2D, Dropout
from keras import Sequential
from keras.applications.vgg16 import VGG16

vgg = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

for layer in vgg.layers[:-4]:
    layer.trainable = False

model = Sequential()

model.add(
    TimeDistributed(vgg, input_shape=(5, 224, 224, 3))
)

model.add(
    TimeDistributed(
        Flatten()
    )
)
model.add(LSTM(256, activation='relu', return_sequences=False))

model.add(Dense(64, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1, activation='linear'))
model.compile(optimizer = 'adam', loss='mean_absolute_error')

In [ ]:
model = keras.models.load_model('C:/e/CRUPA/main_model/e35_val33.881947.h5')

In [ ]:
y_pred = []
y_true = []
for i in range(0,100,5):
    
    img_sample = images_list[i]
    img_sample = np.expand_dims(img_sample, axis=0)
    pred = model.predict(img_sample)
    y_pred.append(pred)
    y_true.append(images_info_df['mass'][i])
    print(images_info_df['mass'][i], '-true')
    print(pred, '- predict')

In [ ]:
df = pd.DataFrame({'y_predict':y_pred, 'y_true':y_true})
df.to_csv('C:/e/CRUPA/table2.csv')

In [3]:
res_df = pd.read_csv('C:/e/CRUPA/table2.csv', sep=',')
res_df['y_predict'] = res_df['y_predict'].apply(lambda x: round(float(x.strip('][')), 1))
res_df['error'] = round(res_df['y_predict'] - res_df['y_true'], 1)
res_df['error_in_%'] = round((res_df['y_predict'] - res_df['y_true']) * 100 / res_df['y_true'], 0)

In [4]:
res_df['error_in_%']

0       -3.0
1       -9.0
2      -76.0
3      -79.0
4       -4.0
5       14.0
6       27.0
7       32.0
8       34.0
9       26.0
10      37.0
11      40.0
12    1638.0
13    1473.0
14    1464.0
15    1471.0
16    1469.0
17    1607.0
18    1412.0
19    1710.0
Name: error_in_%, dtype: float64